In [2]:
import numpy as np
import soccerdata as sd
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt
import json

[10/26/22 09:52:04] INFO     No custom team name replacements found. You can configure these in       ]8;id=981423;file://C:\Users\Saman\anaconda3\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=143496;file://C:\Users\Saman\anaconda3\lib\site-packages\soccerdata\_config.py#85\85]8;;\
                             C:\Users\Saman\soccerdata\config\teamname_replacements.json.                          

                    INFO     No custom league dict found. You can configure additional leagues in    ]8;id=383162;file://C:\Users\Saman\anaconda3\lib\site-packages\soccerdata\_config.py\_config.py]8;;\:]8;id=606070;file://C:\Users\Saman\anaconda3\lib\site-packages\soccerdata\_config.py#155\155]8;;\
                             C:\Users\Saman\soccerdata\config\league_dict.json.                                    

In [4]:
elo = sd.ClubElo()
current_elo = elo.read_by_date()
current_elo.head()

[10/26/22 09:53:09] INFO     Saving cached data to C:\Users\Saman\soccerdata\data\ClubElo             ]8;id=794471;file://C:\Users\Saman\anaconda3\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=552990;file://C:\Users\Saman\anaconda3\lib\site-packages\soccerdata\_common.py#89\89]8;;\

,rank,country,level,elo,from,to,league
team,,,,,,,
Man City,1.0,ENG,1,2037.803833,2022-10-26,2022-10-29,ENG-Premier League
Real Madrid,2.0,ESP,1,2001.127319,2022-10-26,2022-10-30,ESP-La Liga
Liverpool,3.0,ENG,1,1984.683105,2022-10-23,2022-10-26,ENG-Premier League
Bayern,4.0,GER,1,1970.814697,2022-10-23,2022-10-26,GER-Bundesliga
Arsenal,5.0,ENG,1,1911.506714,2022-10-24,2022-10-27,ENG-Premier League


In [23]:
#season_list = [str(i) for i in range(2016,2020)]
season_list = season_list = [str(i) for i in range(2016,2020)]
five38 = sd.FiveThirtyEight('ESP-La Liga', season_list)
games = five38.read_games()

[10/26/22 09:56:48] INFO     Saving cached data to C:\Users\Saman\soccerdata\data\FiveThirtyEight     ]8;id=646159;file://C:\Users\Saman\anaconda3\lib\site-packages\soccerdata\_common.py\_common.py]8;;\:]8;id=154937;file://C:\Users\Saman\anaconda3\lib\site-packages\soccerdata\_common.py#89\89]8;;\

In [26]:
i = 0
GameDates = list(games['date'].values)
GameDates = pd.to_datetime(GameDates)
current_elo = elo.read_by_date(i)
current_elo = elo.read_by_date(pd.to_datetime(list(games['date'].values)[0]).date())

In [106]:
def elo_retriever(date,home_team,away_team):
    #query for current elo
    #current_elo = elo.read_by_date(pd.to_datetime(list(games['date'].values)[0]).date())
    current_elo = elo.read_by_date(date)
    #Try and check if the team is in the wrong name list
    name_dict = {'Deportivo La Coruña':'Depor','Málaga':'Malaga','Real Betis':'Betis','Sevilla FC':'Sevilla','Atletico Madrid':'Atletico','Alavés':'Alaves','Real Sociedad':'Sociedad','Sporting Gijón':'Gijon','Athletic Bilbao':'Bilbao','Celta Vigo':'Celta','Girona FC':'Girona','Real Valladolid':'Valladolid','SD Huesca':'Huesca'}
    if home_team in name_dict.keys():
        home_team = name_dict[home_team]
    if away_team in name_dict.keys():
        away_team = name_dict[away_team]
    #adapt to situation
    home_elo = current_elo.loc[home_team]['elo']
    away_elo = current_elo.loc[away_team]['elo']
    #elos list
    elo_dict = {'home_team':home_elo,'away_team':away_elo}
    return elo_dict

In [107]:
games['date'].dt.date

league       season  game                                 
ESP-La Liga  1617    2016-08-19 Deportivo La Coruña-Eibar     2016-08-19
                     2016-08-19 Málaga-Osasuna                2016-08-19
                     2016-08-20 Barcelona-Real Betis          2016-08-20
                     2016-08-20 Granada-Villarreal            2016-08-20
                     2016-08-20 Sevilla FC-Espanyol           2016-08-20
                                                                 ...    
             1920    2020-07-19 Levante-Getafe                2020-07-19
                     2020-07-19 Osasuna-Mallorca              2020-07-19
                     2020-07-19 Real Valladolid-Real Betis    2020-07-19
                     2020-07-19 Sevilla FC-Valencia           2020-07-19
                     2020-07-19 Villarreal-Eibar              2020-07-19
Name: date, Length: 1520, dtype: object

In [108]:
def result_match(date_calendar,home_team,away_team):
    #re-write date
    home_score = games[(games['date'].dt.date == date_calendar ) & (games['away_team'] == away_team) & (games['home_team'] == home_team)]['score_home']
    away_score = games[(games['date'].dt.date == date_calendar ) & (games['away_team'] == away_team) & (games['home_team'] == home_team)]['score_away']
    #compute difference
    dif = home_score - away_score
    dif = dif.values[0]
    #by default, result is draw
    res = 'D'
    if dif > 0:
        res = 'W'
    elif dif < 0:
        res = 'L'
    return res

In [109]:
GameDates = list(games['date'].values)
GameDates = pd.to_datetime(GameDates)
GameTeamHome = list(games['home_team'].values)
GameTeamAway = list(games['away_team'].values)

In [110]:
home_elos = []
away_elos = []
result = []
for i in range(0,len(GameDates)):
    date_i = GameDates[i]
    date_2_i = date_i.date()
    home_i = GameTeamHome[i]
    away_i = GameTeamAway[i]
    elo_i = elo_retriever(date_i,home_i,away_i)
    home_elos.append(elo_i['home_team'])
    away_elos.append(elo_i['away_team'])
    result.append(result_match(date_2_i,home_i,away_i))

In [116]:
df=pd.DataFrame(data={'Result':result,'Home_Elo':home_elos,'Away_Elo':away_elos})
df.to_csv('/Users/Saman/OneDrive/Desktop/ESP-League-2016-2020.csv')